In [23]:
from pandas.compat.pyarrow import pa
import gensim
import numpy as np
from collections import deque
import random
import re
import pandas as pd
from urllib.parse import urlparse
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Sample dataset of URLs (phishing and legitimate)
p = pd.read_csv('/content/online-valid.csv')
l = pd.read_csv('/content/Benign_list_big_final.csv')

In [3]:
df_p = p.sample(35000)
df_p = df_p['url']
# df_l = l.sample(5000)
df_l = np.array(l)
df_l = df_l.reshape(1,-1)

In [4]:
df_p = np.array(df_p)
df_p

array(['https://diriect.smtib-japan.sbs',
       'https://ipfs-stg.fleek.co/ipfs/QmSec2jtTycoM6oFVg4gbt1hE6xGREMCRn2CaEQi3U62pA?',
       'https://wtfdqcjnjs.duckdns.org/', ..., 'https://s-id-spk.com/',
       'https://docusignaturessystemmer.z15.web.core.windows.net/',
       'http://www.amazceozm-co-jp.amazzeocn.bhouhje2qwhhjyz8.shop/'],
      dtype=object)

In [5]:
df_l

array([['http://1337x.to/torrent/1110018/Blackhat-2015-RUSSIAN-720p-WEB-DL-DD5-1-H264-RUFGT/',
        'http://1337x.to/torrent/1122940/Blackhat-2015-x264-1080p-WEB-DL-eng-nl-subs-sharky/',
        'http://1337x.to/torrent/1124395/Fast-and-Furious-7-2015-HD-TS-XVID-AC3-HQ-Hive-CM8/',
        ...,
        'https://lastpass.com/signup2.php?ac=1&from_uri=%2Fsignup2.php%3Fac%3D1%26from_uri%3D%252Fsignup2.php%253Fac%253D1%2526from_uri%253D%25252Fsignup2.php%25253Fac%25253D1%252526from_uri%25253D%2525252Fsignup2.php%2525253Fac%2525253D1%25252526from_uri%2525253D%252525252Fsignup2.php%252525253Fac%252525253D1%2525252526from_uri%252525253D%25252525252Fsignup2.php%25252525253Fac%25252525253D1%252525252526from_uri%25252525253D%2525252525252Fsignup2.php%2525252525253Fac%2525252525253D1%25252525252526from_uri%2525252525253D%252525252525252Fsignup2.php%252525252525253Fac%252525252525253D1%2525252525252526from_uri%252525252525253D%25252525252525252Fsignup2.php%25252525252525253Fac%25252525252525253D

In [7]:
# Preprocess URLs into lists of words (tokenization)
def preprocess_url(url):
    parsed_url = url.split('/')
    return parsed_url

In [8]:
phishing_urls_preprocessed = [preprocess_url(url) for url in df_p]
legitimate_urls_preprocessed = [preprocess_url(url) for url in df_l[0]]

In [9]:
phishing_urls_preprocessed

[['https:', '', 'diriect.smtib-japan.sbs'],
 ['https:',
  '',
  'ipfs-stg.fleek.co',
  'ipfs',
  'QmSec2jtTycoM6oFVg4gbt1hE6xGREMCRn2CaEQi3U62pA?'],
 ['https:', '', 'wtfdqcjnjs.duckdns.org', ''],
 ['https:',
  '',
  'bafybeidfxihuewos2x2mwmf6ruquskfup5s4im6yuyhthw5ra5yhvydk2u.ipfs.cf-ipfs.com',
  ''],
 ['https:', '', 'rhtdwbx-ajb-7.tk', 'skking89', 'office', 'index.html'],
 ['https:', '', 'dev-roepaporoa.pantheonsite.io', ''],
 ['https:',
  '',
  'gateway.ipfs.io',
  'ipfs',
  'bafybeihmwtc3dcvi7qsiv6hqq42gxfypnmawndi6tkm6wwuufxom4lphte'],
 ['https:', '', 'hotmail-104408.weeblysite.com', ''],
 ['https:',
  '',
  'bafybeib2gf24er55gzcqdyxuysgvho2vzvohv3kx6dk6ynsw5dh2wxgywa.ipfs.cf-ipfs.com',
  'pl2auth.html'],
 ['https:',
  '',
  'spexi-spexianas9553677.codeanyapp.com',
  'switzz',
  's',
  'de',
  'username.html'],
 ['https:', '', 'hfkf6-3aaaa-aaaad-qewra-cai.raw.ic0.app', ''],
 ['https:', '', 'veytb6.webwave.dev', ''],
 ['http:', '', '47u2vjok.duckdns.org'],
 ['https:',
  '',
  'bafyb

In [10]:
# Train Word2Vec model
all_urls = phishing_urls_preprocessed + legitimate_urls_preprocessed
word2vec_model = gensim.models.Word2Vec(all_urls, vector_size=100, window=5, min_count=1, sg=0)

In [11]:
legitimate_urls_preprocessed

[['http:',
  '',
  '1337x.to',
  'torrent',
  '1110018',
  'Blackhat-2015-RUSSIAN-720p-WEB-DL-DD5-1-H264-RUFGT',
  ''],
 ['http:',
  '',
  '1337x.to',
  'torrent',
  '1122940',
  'Blackhat-2015-x264-1080p-WEB-DL-eng-nl-subs-sharky',
  ''],
 ['http:',
  '',
  '1337x.to',
  'torrent',
  '1124395',
  'Fast-and-Furious-7-2015-HD-TS-XVID-AC3-HQ-Hive-CM8',
  ''],
 ['http:',
  '',
  '1337x.to',
  'torrent',
  '1145504',
  'Avengers-Age-of-Ultron-2015-CAM-New-Audio-x264-CPG',
  ''],
 ['http:',
  '',
  '1337x.to',
  'torrent',
  '1160078',
  'Avengers-age-of-Ultron-2015-HQ-CAM-H264-AC3-MURD3R',
  ''],
 ['http:',
  '',
  '1337x.to',
  'torrent',
  '294349',
  'American-Idol-S11E04-Auditions-4-HDTV-XviD-FQM-ettv',
  ''],
 ['http:',
  '',
  '189.cn',
  'dqmh',
  'userCenter',
  'myOrderInfoList.do?method=listMyOrderInfo_new&isVs=no'],
 ['http:',
  '',
  '2gis.ru',
  'moscow',
  'search',
  '%D0%9F%D0%BE%D0%B5%D1%81%D1%82%D1%8C',
  'tab',
  'firms',
  'zoom',
  '11'],
 ['http:',
  '',
  'abc.go.com

In [12]:
# Function to generate Word2Vec embeddings for URLs
def url_to_word_embeddings(url):
    embeddings = []
    for word in url:
        if word in word2vec_model.wv:
            embeddings.append(word2vec_model.wv[word])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Return zeros if no valid embeddings

In [13]:
# Combine preprocessed URLs and labels
X = [url_to_word_embeddings(url) for url in phishing_urls_preprocessed + legitimate_urls_preprocessed]
y = [1] * len(phishing_urls_preprocessed) + [0] * len(legitimate_urls_preprocessed)

In [27]:
dfx = pd.DataFrame(X)
dfy= pd.DataFrame(y)

In [26]:
dfx

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.444467,0.718474,-0.085958,1.258038,-0.534685,0.284074,0.150893,1.133715,-1.366511,-0.947633,...,0.520407,-0.528113,1.228445,-0.910764,1.907892,0.959495,0.411711,-0.476249,-0.403848,0.181884
1,-0.435136,0.588301,-0.295598,0.954597,-0.113596,0.341944,0.059858,1.251485,-1.182166,-0.644718,...,0.653847,-0.411402,1.335061,-0.748646,1.757730,0.688504,0.346738,-0.242000,-0.204934,0.257339
2,-0.626787,0.811946,0.128399,1.530964,-0.867718,0.164851,0.301107,1.253105,-1.536918,-1.193367,...,0.574784,-0.670335,1.193761,-1.176497,2.315438,1.113639,0.388076,-0.766914,-0.560565,0.423921
3,-0.625664,0.811658,0.131341,1.534174,-0.871059,0.164869,0.301467,1.257463,-1.537480,-1.195590,...,0.573509,-0.669854,1.194571,-1.176990,2.319149,1.111518,0.389188,-0.768012,-0.560592,0.421255
4,-0.288106,0.421879,-0.069524,0.783039,-0.403844,0.156500,0.034357,0.784309,-0.844925,-0.480894,...,0.440404,-0.249196,0.885024,-0.559331,1.228104,0.638336,0.195953,-0.183930,-0.241156,0.191871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70372,-0.342700,0.524611,-0.079070,0.968030,-0.441200,0.210238,0.095329,0.882350,-1.031712,-0.699665,...,0.419219,-0.406000,0.969282,-0.699358,1.462769,0.744250,0.302033,-0.329744,-0.299241,0.153371
70373,-0.341889,0.520978,-0.078039,0.968675,-0.434283,0.208326,0.097624,0.881788,-1.025210,-0.698595,...,0.416743,-0.406540,0.960177,-0.696361,1.452908,0.741263,0.297012,-0.330389,-0.296459,0.153047
70374,-0.343017,0.524269,-0.079937,0.968978,-0.440445,0.212620,0.099251,0.884134,-1.031788,-0.698985,...,0.417715,-0.404775,0.966146,-0.699805,1.458159,0.744614,0.299786,-0.330439,-0.297382,0.152891
70375,-0.344442,0.524180,-0.079964,0.969151,-0.437051,0.211024,0.096479,0.883787,-1.029602,-0.694968,...,0.418331,-0.404661,0.968670,-0.701362,1.459857,0.744389,0.302171,-0.329447,-0.296608,0.153580


In [28]:
dfy.to_csv('y.csv')
dfx.to_csv("X.csv")